In [47]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import ng_models
import matplotlib.pyplot as plt
from collections import Counter
import stanza

#stanza.download('pl')

%matplotlib notebook
importlib.reload(funcs)
importlib.reload(ng_models)

stopwords = open('00_dane/polish.stopwords.txt','r').read().split()
stopwords.append(['.', ',','!', '?'])
stopwords.append(',')

In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined2.csv',  sep='|', encoding='UTF-8')

## Przygotowanie zbioru danych pod analizę tematyczną (i poźniej pod analizę sentymentu również):
- lematyzacja
- usunięcie stop words
- wektoryzacja

In [9]:
sp_joined.head(1)

,Unnamed: 0,date_,author_final,speech_title,speech_raw,speech_rawer,liczba_sl_raw,liczba_sl_rawer,full_name,term,party_final
0,0,01-02-1992,Aleksander Bentkowski,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoki Sejmie! Chciałbym kil...,Chciałbym kilka zdań powiedzieć na temat skut...,851,845,Aleksander Bentkowski,3.0,Polskie Stronnictwo Ludowe


In [12]:
sp = sp_joined[['date_', 'speech_rawer', 'full_name', 'party_final']]

In [58]:
sp.head(10)

,date_,speech_rawer,full_name,party_final,speech_rawest
0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,chcieć zdanie powiedzieć temat skutek ustawa ....
1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,przysłuchiwać emocjonalny długo trwać debata o...
2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,różny przebyć droga powiedzieć swój oczekiwani...
3,01-02-1992,"Wszystkim nam bardzo zależy na tym, aby urato...",Alojzy Szablewski,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,wszystek zależeć uratować przemysł . stoczeń G...
4,01-02-1992,Z brzmienia pkt. 1 projektu uchwały odczytuję...,Andrzej Baraniecki,Sojusz Lewicy Demokratycznej,brzmię punkt . 1 projekt uchwała odczytować se...
5,01-02-1992,W związku z tym chcę zapytać panią Ponieważ S...,Andrzej Potocki,Klub Parlamentarny Unii Wolności \r\n,związek chcieć zapytać sejm uchwalać prawo uch...
6,01-02-1992,"Andrzej Tadeusz Mazurkiewicz, Konfederacja Po...",NaN,NaN,Andrzej Tadeusz Mazurkiewicz konfederacja Pols...
7,01-02-1992,Parlamentarny Klub Porozumienia Centrum pop...,Andrzej Urbański,Klub Polski Program Liberalny,parlamentarny klub porozumienie centrum popier...
8,01-02-1992,W nocy z 12 na 13 grudnia 1981 r. ówczesne ko...,Andrzej Wojtyła,Koło Poselskie Konserwatywno-Ludowe\r\n,noc 12 13 grudzień 1981 rok ówczesny komunisty...
9,01-02-1992,Klub Liberalno-Demokratyczny popiera projekt ...,Andrzej Zarębski,Klub Polski Program Liberalny,klub liberalny - demokratyczny popierać projek...


### Usunięcie stop words oraz lematyzacja

In [ ]:
nlp = stanza.Pipeline('pl')

In [ ]:
%%time
x=0
for i in range(len(sp['speech_rawer'])):
    
    doc = nlp(sp.loc[i, 'speech_rawer'])
    sp.loc[i, 'speech_rawest']= ' '.join([word.lemma for sent in doc.sentences for word in sent.words if word.lemma not in stopwords])
    if round(100*i/len(sp['speech_rawer']),0)>x:
        x = round(100*i/len(sp['speech_rawer']),0)
        print('Done: ',x,'%')

C:\Users\Magda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Done:  1.0 %
Done:  2.0 %


In [49]:
funcs.speech_SW_LEM(sp['speech_rawer'][0])


2020-04-26 13:35:08 INFO: Loading these models for language: pl (Polish):
| Processor | Package |
-----------------------
| tokenize  | lfg     |
| pos       | lfg     |
| lemma     | lfg     |
| depparse  | lfg     |

2020-04-26 13:35:08 INFO: Use device: cpu
2020-04-26 13:35:08 INFO: Loading: tokenize
2020-04-26 13:35:08 INFO: Loading: pos
2020-04-26 13:35:08 INFO: Loading: lemma
2020-04-26 13:35:09 INFO: Loading: depparse
2020-04-26 13:35:09 INFO: Done loading processors!


NameError: name 'stopwords' is not defined

In [31]:
dicts

[[{'id': '1',
   'text': 'Chciał',
   'lemma': 'chcieć',
   'upos': 'VERB',
   'xpos': 'praet:sg:m1:imperf',
   'feats': 'Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|SubGender=Masc1|Tense=Past|VerbForm=Fin|Voice=Act',
   'head': 0,
   'deprel': 'root',
   'misc': 'start_char=1|end_char=7'},
  {'id': '2',
   'text': 'by',
   'lemma': 'by',
   'upos': 'AUX',
   'xpos': 'qub',
   'feats': 'Mood=Cnd',
   'head': 1,
   'deprel': 'aux:mood',
   'misc': 'start_char=7|end_char=9'},
  {'id': '3',
   'text': 'm',
   'lemma': 'być',
   'upos': 'AUX',
   'xpos': 'aglt:sg:pri:imperf:nwok',
   'feats': 'Aspect=Imp|Number=Sing|Person=1|Variant=Short',
   'head': 1,
   'deprel': 'aux:aglt',
   'misc': 'start_char=9|end_char=10'},
  {'id': '4',
   'text': 'kilka',
   'lemma': 'kilka',
   'upos': 'DET',
   'xpos': 'num:pl:acc:n:rec',
   'feats': 'Case=Acc|Gender=Neut|NumType=Card|Number=Plur|PronType=Ind',
   'head': 5,
   'deprel': 'det',
   'misc': 'start_char=11|end_char=16'},
  {'id': '5',
   'text'